# Engenharia de Atributos

In [44]:
import pandas as pd

In [45]:
merged_dataset = pd.read_csv('../data/processed/merged_dataset.csv')

C:\Users\Matheus\AppData\Local\Temp\ipykernel_20812\3999875462.py:1: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_dataset = pd.read_csv('../data/processed/merged_dataset.csv')


Convertendo as variáveis para os seus respectivos tipos corretos 

In [46]:
# Conversão dos tipos numéricos e categóricos
merged_dataset = merged_dataset.astype({
    'ID_CLIENTE': 'int64',
    'VALOR_A_PAGAR': 'float64',
    'TAXA': 'float64',
    'RENDA_MES_ANTERIOR': 'float64',
    'NO_FUNCIONARIOS': 'int64',
    'FLAG_PF': 'int', 
    'DIAS_ATRASO': 'Int64',
    'TARGET_INADIMPLENCIA': 'int',
    'SEGMENTO_INDUSTRIAL': 'category',
    'PORTE': 'category',
    'CEP_2_DIG': 'category',
    'DIAS_ADIANTAMENTO': 'int',
    'DOMINIO_EMAIL': 'category',
})

# Conversão de datas
merged_dataset['SAFRA_REF'] = pd.to_datetime(
    merged_dataset['SAFRA_REF'], format='%Y-%m-%d', errors='coerce'
)

merged_dataset['DATA_EMISSAO_DOCUMENTO'] = pd.to_datetime(
    merged_dataset['DATA_EMISSAO_DOCUMENTO'], format='%Y-%m-%d', errors='coerce'
)

merged_dataset['DATA_PAGAMENTO'] = pd.to_datetime(
    merged_dataset['DATA_PAGAMENTO'], format='%Y-%m-%d', errors='coerce'
)

merged_dataset['DATA_VENCIMENTO'] = pd.to_datetime(
    merged_dataset['DATA_VENCIMENTO'], format='%Y-%m-%d', errors='coerce'
)

merged_dataset['DATA_CADASTRO'] = pd.to_datetime(
    merged_dataset['DATA_CADASTRO'], format='%Y-%m-%d', errors='coerce'
)


In [47]:
merged_dataset.dtypes

ID_CLIENTE                         int64
SAFRA_REF                 datetime64[ns]
DATA_EMISSAO_DOCUMENTO    datetime64[ns]
DATA_PAGAMENTO            datetime64[ns]
DATA_VENCIMENTO           datetime64[ns]
VALOR_A_PAGAR                    float64
TAXA                             float64
RENDA_MES_ANTERIOR               float64
NO_FUNCIONARIOS                    int64
DATA_CADASTRO             datetime64[ns]
FLAG_PF                            int64
SEGMENTO_INDUSTRIAL             category
DOMINIO_EMAIL                   category
PORTE                           category
CEP_2_DIG                       category
DIAS_ATRASO                        Int64
TARGET_INADIMPLENCIA               int64
DOMINIO_EMAIL_GRUPO               object
DIAS_ADIANTAMENTO                  int64
dtype: object

Criando novas features (pense nelas)

Calculando tempo de casa. Indica a relação dos clientes com a empresa, isso pode influenciar na inadimplência. Clientes mais velhos de casa têm menos tendência a inadimplência.

In [48]:
merged_dataset['TEMPO_CADASTRO_PARA_VENCIMENTO'] = (merged_dataset['DATA_VENCIMENTO'] - merged_dataset['DATA_CADASTRO']).dt.days

In [49]:
merged_dataset['TEMPO_DE_CASA_MESES'] = merged_dataset['TEMPO_CADASTRO_PARA_VENCIMENTO'] // 30


In [50]:
merged_dataset = merged_dataset.drop(columns='TEMPO_CADASTRO_PARA_VENCIMENTO')


Prazo para o pagamento: ajuda a entender o tempo que o cliente teve para se preparar para o pagamento. Prazos curtos podem influenciar na inadimplência.



In [51]:
merged_dataset['PRAZO_PAGAMENTO_DIAS'] = (merged_dataset['DATA_VENCIMENTO'] - merged_dataset['DATA_EMISSAO_DOCUMENTO']).dt.days

Mês/Ano da SAFRA_REF: capta padrões sazonais (ex: pode haver mais inadimplência no final do ano ou em meses com impostos).

In [52]:
merged_dataset['MES_SAFRA'] = merged_dataset['SAFRA_REF'].dt.month
merged_dataset['ANO_SAFRA'] = merged_dataset['SAFRA_REF'].dt.year


Criação de **INADIMPLENCIAS_ANTERIORES**. Essa variável é um contador cumulativo do número de inadimplências passadas para cada cliente até o momento do pagamento atual. Ela captura o comportamento passado do cliente, que é um forte indicador de risco de inadimplência. Clientes reincidentes tendem a ter maior probabilidade de inadimplir novamente.

In [53]:
# Ordena por cliente e data de emissão
merged_dataset = merged_dataset.sort_values(by=['ID_CLIENTE', 'DATA_EMISSAO_DOCUMENTO'])

# Cria coluna cumulativa de inadimplências
merged_dataset['INADIMPLENCIAS_ANTERIORES'] = (
    merged_dataset
    .groupby('ID_CLIENTE')['TARGET_INADIMPLENCIA']
    .cumsum()
    .shift(fill_value=0)  # Para não contar a linha atual
)


In [54]:
merged_dataset.dtypes

ID_CLIENTE                            int64
SAFRA_REF                    datetime64[ns]
DATA_EMISSAO_DOCUMENTO       datetime64[ns]
DATA_PAGAMENTO               datetime64[ns]
DATA_VENCIMENTO              datetime64[ns]
VALOR_A_PAGAR                       float64
TAXA                                float64
RENDA_MES_ANTERIOR                  float64
NO_FUNCIONARIOS                       int64
DATA_CADASTRO                datetime64[ns]
FLAG_PF                               int64
SEGMENTO_INDUSTRIAL                category
DOMINIO_EMAIL                      category
PORTE                              category
CEP_2_DIG                          category
DIAS_ATRASO                           Int64
TARGET_INADIMPLENCIA                  int64
DOMINIO_EMAIL_GRUPO                  object
DIAS_ADIANTAMENTO                     int64
TEMPO_DE_CASA_MESES                   int64
PRAZO_PAGAMENTO_DIAS                  int64
MES_SAFRA                             int32
ANO_SAFRA                       

In [55]:
merged_dataset

,ID_CLIENTE,SAFRA_REF,DATA_EMISSAO_DOCUMENTO,DATA_PAGAMENTO,DATA_VENCIMENTO,VALOR_A_PAGAR,TAXA,RENDA_MES_ANTERIOR,NO_FUNCIONARIOS,DATA_CADASTRO,...,CEP_2_DIG,DIAS_ATRASO,TARGET_INADIMPLENCIA,DOMINIO_EMAIL_GRUPO,DIAS_ADIANTAMENTO,TEMPO_DE_CASA_MESES,PRAZO_PAGAMENTO_DIAS,MES_SAFRA,ANO_SAFRA,INADIMPLENCIAS_ANTERIORES
0,8784237149961904,2018-09-01,2018-09-04,2018-09-24,2018-09-24,59610.76,5.99,300502.0,107,2011-02-14,...,27,0,0,HOTMAIL,0,92,20,9,2018,0
1,8784237149961904,2018-09-01,2018-09-06,2018-09-24,2018-09-24,39398.06,5.99,300502.0,107,2011-02-14,...,27,0,0,HOTMAIL,0,92,18,9,2018,0
2,8784237149961904,2018-09-01,2018-09-09,2018-09-25,2018-09-25,55416.75,5.99,300502.0,107,2011-02-14,...,27,0,0,HOTMAIL,0,92,16,9,2018,0
3,8784237149961904,2018-09-01,2018-09-11,2018-09-27,2018-09-27,11751.35,5.99,300502.0,107,2011-02-14,...,27,0,0,HOTMAIL,0,92,16,9,2018,0
4,8784237149961904,2018-09-01,2018-09-17,2018-10-03,2018-10-03,35985.00,5.99,300502.0,107,2011-02-14,...,27,0,0,HOTMAIL,0,92,16,9,2018,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72485,9206030810342980458,2021-05-01,2021-05-16,2021-06-01,2021-06-01,107321.36,11.99,256133.0,109,2005-08-08,...,91,0,0,HOTMAIL,0,192,16,5,2021,2
72486,9206030810342980458,2021-05-01,2021-05-23,2021-06-08,2021-06-08,38372.91,5.99,256133.0,109,2005-08-08,...,91,0,0,HOTMAIL,0,192,16,5,2021,2
72487,9206030810342980458,2021-06-01,2021-06-16,2021-07-02,2021-07-02,43030.50,5.99,463963.0,105,2005-08-08,...,91,0,0,HOTMAIL,0,193,16,6,2021,2
72488,9206030810342980458,2021-06-01,2021-06-18,2021-07-05,2021-07-05,107318.81,5.99,463963.0,105,2005-08-08,...,91,0,0,HOTMAIL,0,193,17,6,2021,2


Pré-processamento dos dados para o modelo 

Colunas a serem removidas:

- ID_CLIENTE: funciona como um identificador, não possui informação útil para predição.
- DATA_EMISSAO_DOCUMENTO, DATA_PAGAMENTO, DATA_VENCIMENTO, DATA_CADASTRO, SAFRA_REF: São datas brutas. Foram transformadas em variáveis derivadas.


Colunas categóricas a serem tratadas: 
- SEGMENTO_INDUSTRIAL, PORTE, CEP_2_DIG, DOMINIO_EMAIL: aplicar One-Hot Encoding 



Aplicando o pré-processamento 

In [ ]:
# Eliminar colunas irrelevantes ou de alta cardinalidade
cols_drop = [
    'ID_CLIENTE', 'DATA_EMISSAO_DOCUMENTO', 'DATA_PAGAMENTO',
    'DATA_VENCIMENTO', 'DATA_CADASTRO', 'SAFRA_REF'
]

In [ ]:


#merged_dataset.to_csv('../data/processed/train_merged_dataset.csv', index=False)